In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

In [ ]:
alldf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'operator.parquet'))
# print(alldf.columns)
# alldf.date = pd.to_datetime(alldf.date)
alldf['year'] = alldf.date.dt.year
alldf = alldf[alldf.year>2010]  # don't include historic data
opname = alldf.bgOperatorName.iloc[0]
commname = alldf.OperatorName.value_counts().index[0]

In [ ]:
showHeader(f'{commname}',f'({opname})', link_up_level=1)

In [ ]:
disc_w_ing = len(alldf[~alldf.no_chem_recs].DisclosureId.unique())
disc_w_water = len(alldf[alldf.TotalBaseWaterVolume>0].DisclosureId.unique())
display(md(f'### Total number of disclosures : {len(alldf.DisclosureId.unique()):,}'))
display(md(f'### Number of disclosures with water volume: {disc_w_water:,}'))
display(md(f'### Number of disclosures with chemical records: {disc_w_ing:,}'))


In [ ]:
display(md(f'## Disclosure locations: {opname.upper()}'))

In [ ]:
def xlate_val(n):
    if n==0:
        return ''
    if n<1000:
        return round_sig(n,1)
    x = round_sig(n,1)
    return x[0]+ 'k'

def make_annot(gb):
    annot = gb.copy()
    annot.DisclosureId = annot.DisclosureId.map(lambda x: xlate_val(x))
    #print(annot)
    piv = annot.pivot(index='County',columns='year',values='DisclosureId')
    piv.fillna('',inplace=True)
    #print(piv)
    return piv

def get_zoom_level(df):
    latdiff = df.bgLatitude.max() - df.bgLatitude.min()
    londiff = df.bgLongitude.max()- df.bgLongitude.min()
    #print(f'latdiff = {latdiff}, londiff = {londiff}')
    diffsum = latdiff+londiff
    if diffsum <1 : return 6
    if diffsum <5 : return 5
    if diffsum <20 : return 4
    if diffsum <28 : return 3.5
    return 3
def get_geog_center(state_list):
    t = pd.read_csv(r"C:\MyDocs\OpenFF\src\openFF-catalog\work\state_coords.csv",
                   dtype={'Latitude':'float', 'Longitude':'float'})
    t = t[t.state.isin(state_list)]
    #print(t)
    return [t.Latitude.mean(),t.Longitude.mean()*-1]

def CountyMap(df):
    state_list = df.bgStateName.unique().tolist()
    start_loc = get_geog_center(state_list)
    #print(statename,start_loc)
    cond = ~df.location_error
    if cond.sum()==0:  # no valid fracks for this state
        display(md('## No mappable fracks for this operator!'))
        # display(md(f'Any data in this state set may be labeled incorrectly as {statename}'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'DisclosureId'],as_index=False).size()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'DisclosureId':'value'},
                                                                                                axis=1)
    zoom = get_zoom_level(df[['bgLatitude','bgLongitude']])
    mapping.create_county_choropleth(gb,plotlog=False,#plotlog=True,custom_scale= [0,1,2,3,4],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value'],
                             aliases=['State: ','County: ','Number Fracking disclosures: '])

def PointMap(df):
    gb = df.groupby('DisclosureId')[['bgLatitude','bgLongitude','APINumber','TotalBaseWaterVolume',
                                 'year','OperatorName','ingKeyPresent']].first()
    gb['year'] = gb.year.astype('str')
    gb.TotalBaseWaterVolume = gb.TotalBaseWaterVolume.map(lambda x: round_sig(x,3,guarantee_str='??')) + ' gallons'
    gb.APINumber = gb.APINumber.astype('str')
    # gb.drop('date',axis=1,inplace=True)
    create_point_map(gb)
    
    

### ...by county
Note that not all fracks have valid County information and so will not show up in the following map (especially Alaska).

In [ ]:
CountyMap(alldf)

### ...by individual wells
Use the layer controls in the upper right to turn on and off the types of markers as well as showing the map or satellite views.  Cluster markers are best for big picture view, information markers include popups with a bit of info about the well.  Note that for operators with a large number of disclosures, the information markers take a few seconds to update.

In [ ]:
# def make_disc_link(row):
#     return getDisclosureLink(row.APINumber,row.DisclosureId,row.APINumber)

# alldf.APINumber = alldf.apply(lambda x: make_disc_link(x),axis=1)
# print(alldf.columns)
mapping.create_integrated_point_map(alldf[['DisclosureId','bgLatitude','bgLongitude','APINumber',
                                           'WellName','TotalBaseWaterVolume',
                                           'OperatorName','year','no_chem_recs']],
                 aliases=['API Number:','Well Name','Water used (gallons):','Operator:','date:','Chem recs only in PDF:'],
                 fields=['APINumber','WellName','TotalBaseWaterVolume','OperatorName','year','no_chem_recs'],
                 # include_shape = shape_flag,
                 # area_df = geojson
                )

# Active Years by state

In [ ]:
gb = alldf.groupby(['bgStateName','DisclosureId','year'],as_index=False).size()
gb1 = gb.groupby(['bgStateName','year'],as_index=False).size().rename({'size':'num_disc'},axis=1)
gb1.bgStateName = gb1.bgStateName.str.title()
nstates = len(gb1.bgStateName.unique())
dim = 4
if nstates>12:
    dim = 5
if nstates>15:
    dim = 6
pd.pivot_table(gb1,index='year',columns='bgStateName',values='num_disc').plot.bar(subplots=True,
                                                                                 layout=(dim,3),
                                                                                 legend=False,
                                                                                 figsize=(10,10),
                                                                                 title='Number of disclosures');
# for state in states:
#     gb1[gb1.bgStateName==state].plot.bar('year','num_disc',title=f'{state.title()}',figsize=(3,2),
#                                          ylabel='Number of disclosures',legend=False)

# Chemicals of Concern
Percent of this company's disclosures with chemicals on the following lists:
- Clean Water Act list
- Safe Drinking Water Act list
- California's Prop. 65 list
- TSCA's "UVCB" list - "Unknown, variable composition..."
- EPA's PFAS master list
- EPA's diesel list (4 chemicals)

In [ ]:
gb= alldf.groupby('DisclosureId',as_index=False)[['is_on_CWA','is_on_DWSHA','is_on_PFAS_list',
                                             'is_on_prop65',
                                             'is_on_UVCB','is_on_diesel']].count()
CWA = len(gb[gb.is_on_CWA>0])/disc_w_ing *100
DWSHA = len(gb[gb.is_on_DWSHA>0])/disc_w_ing *100
PFAS = len(gb[gb.is_on_PFAS_list>0])/disc_w_ing *100
# volatile = len(gb[gb.is_on_volatile_list>0])/disc_w_ing *100
prop65 = len(gb[gb.is_on_prop65>0])/disc_w_ing *100
UVCB = len(gb[gb.is_on_UVCB>0])/disc_w_ing *100
diesel = len(gb[gb.is_on_diesel>0])/disc_w_ing *100
t = pd.DataFrame({'list':['CWA','DWSHA','prop65','UVCB','PFAS','diesel'],
                  'Percent_disc':[CWA,DWSHA,prop65,UVCB,PFAS,diesel]})
ax = t.plot.bar('list','Percent_disc',legend=False,ylabel='% of disclosures with at least\n one chemical on list')
plt.xticks(rotation=0);

# Water use
Reported `TotalBaseWaterVolume` on the disclosure (in gallons).

In [ ]:
totpres = len(alldf.DisclosureId.unique())

if totpres<300:
    alpha = 1
elif totpres<2000:
    alpha = .6
elif totpres<20000:
    alpha = .35
else:
    alpha = .2
    
min_water = 10 # number disclosures needed to trigger graphs

In [ ]:
if (disc_w_water>min_water):
    gb = alldf.groupby('DisclosureId',as_index=False)[['TotalBaseWaterVolume','date']].first()
    cond = (gb.TotalBaseWaterVolume>0) & (gb.date.dt.year>2010)
    gb['not_present'] = gb[cond].TotalBaseWaterVolume.min() - (gb[cond].TotalBaseWaterVolume.max()-gb[cond].TotalBaseWaterVolume.min())*0.05
    ax = gb[cond].plot('date','TotalBaseWaterVolume', style='o', alpha=alpha,
                figsize=(16,6),legend=False)
    gb[~cond].plot('date','not_present', style='|', alpha=1,color='orange',ms=20, ax=ax,legend=False)
    plt.ylabel('Gallons reported',fontsize=16);
    plt.title(f'Gallons of water used for each job: {opname.upper()} -- linear version',fontsize=16);
    ax.grid()
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));
    display(md(f'##### Number of disclosures with valid water volume (shown by blue circles): {len(gb[cond]):,}\n'))
    display(md(f'##### Number of records without valid water volume (shown by orange bars): {len(gb[~cond]):,}\n'))


In [ ]:
# if alldf.TotalBaseWaterVolume.max()>0:
#     display(md(f'##### Number of disclosures with valid water volume (shown by blue circles): {len(gb[cond]):,}\n'))
#     display(md(f'##### Number of records without valid water volume (shown by orange bars): {len(gb[~cond]):,}\n'))
#     display(md('---'))

In [ ]:
if (disc_w_water>min_water):
    ax = gb[cond].plot('date','TotalBaseWaterVolume', style='o', alpha=alpha,
                figsize=(16,6))
    plt.ylabel('Gallons reported',fontsize=16);
    plt.title(f'Gallons of water used for each job: {opname.upper()} -- log version',fontsize=16);
    ax.set(yscale='log')
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    if len(gb[cond]) < 5000: # provide more detailed grid (too many points swamps it out)
        locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    else:
        locmaj = matplotlib.ticker.LogLocator(base=10) #,subs='all') 
    ax.yaxis.set_major_locator(locmaj)
    ax.set(ylim=(max(10,gb.TotalBaseWaterVolume.min()),
                 gb.TotalBaseWaterVolume.max()*1.4));
    ax.grid()

    lns = list(np.percentile(gb[cond].TotalBaseWaterVolume,[25,50,75]))
    #ax.set_ylim(-0.7,len(sn)-0.3)
    for l in lns:
        plt.hlines(l,gb[cond].date.min(),
                   gb[cond].date.max(),
                   color='black')
    s = 'PERCENTILES:\n'
    s+= ' -- 25%:  {:,} gallons\n'.format(float(lns[0]))
    s+= ' -- 50%:  {:,} gallons\n'.format(float(lns[1]))
    s+= ' -- 75%:  {:,} gallons\n'.format(float(lns[2]))
    s+= ' -- max:  {:,} gallons\n'.format(float(gb[cond].TotalBaseWaterVolume.max()))
    print(s)

In [ ]:
if (disc_w_water<=min_water)&(alldf.TotalBaseWaterVolume.max()>0):
    gb = alldf[alldf.TotalBaseWaterVolume>0].groupby('DisclosureId',as_index=False)[['date','APINumber',
                                                                   'TotalBaseWaterVolume',
                                                                   'bgStateName','bgCountyName']].first()
    iShow(gb[['date','APINumber','TotalBaseWaterVolume','bgStateName','bgCountyName']])

In [ ]:
# if disc_w_ing>0:
#     gb1 = alldf.groupby('bgSupplier',as_index=False)[['DisclosureId']].nunique().rename({'DisclosureId':'disclosure_cnt'},axis=1)
#     # gb1 = gb.groupby('bgSupplier',as_index=False)['DisclosureId'].count()
#     gb2 = alldf.groupby('bgSupplier')['Supplier'].agg(lambda x: x.value_counts().index[0])
#     gb2 = gb2.reset_index()
#     gb2.columns = ['bgSupplier','most_common_name']
#     mg = pd.merge(gb2,gb1,on='bgSupplier',how='outer')
#     iShow(mg)
# else:
#     display(md('>### **No disclosures with chemical records for this operator**'))

In [ ]:
from matplotlib.offsetbox import AnchoredText

def proprietary_bars(df,plot_title='TEST',minyr=2011,maxyr=2021):
    df = df.copy()
    df['propCut'] = pd.cut(df.perc_proprietary,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary claims','up to 10% \nare proprietary claims',
                                  'between 10 and 25% \nare proprietary claims',
                                  'between 25 and 50% \nare proprietary claims',
                                  'greater than 50% \nare proprietary claims'])
    
    import seaborn as sns
    t = df.propCut.value_counts(sort=False).reset_index()
    ax = sns.barplot(data=t,y='index',x='propCut',palette='Reds',orient="h")
    ax.set_xlabel("Number of disclosures")
    ax.set_ylabel("")
    ax.set_title(plot_title)
    ax.invert_yaxis()


if disc_w_ing > 0:
    display(md("# Proprietary claims"))
    testtitle = opname.upper() +': Trade Secret frequency'
    # print(alldf.columns)
    proprietary_bars(alldf,testtitle,
                     minyr=max(2011,alldf.date.dt.year.min()),
                     maxyr=alldf.date.dt.year.max())
